In [1]:
## Get the raw text file
import os
import requests

if not os.path.exists("the-verdict.txt"):
    url = (
        "https://raw.githubusercontent.com/rasbt/"
        "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
        "the-verdict.txt"
    )
    file_path = "the-verdict.txt"

    response = requests.get(url, timeout=30)
    response.raise_for_status()
    with open(file_path, "wb") as f:
        f.write(response.content)

## Section 2.2: Tokenizing text

In [2]:
with open('the-verdict.txt','r',encoding='utf-8') as f:
    raw_text = f.read()
print('Total number of characters:', len(raw_text))
print(raw_text[:99])

Total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [3]:
import re

text = "Hello world! This is a test."

#split
result = re.split(r'\s',text)
print(result)

#split using punctuation
result = re.split(r'([,.]|\s)',text)
print(result)

#remove empties
result = [item for item in result if item.strip()]
print(result)

#more general punctuation issues:
text = 'Hello, world. Is this-- a test?'

result = re.split(r'([,.:;?_!"()\']|--|\s)',text)
result = [item for item in result if item.strip()]
print(result)



['Hello', 'world!', 'This', 'is', 'a', 'test.']
['Hello', ' ', 'world!', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']
['Hello', 'world!', 'This', 'is', 'a', 'test', '.']
['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [4]:
#now preprocess our raw text file
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item for item in preprocessed if item.strip()]
print(preprocessed[:30])
print(len(preprocessed))

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']
4690


In [5]:
#build vocabulary
all_words = sorted(set(preprocessed))    
vocab_size = len(all_words)
print('Vocab size:',vocab_size)

#assign numeric ids to words
vocab = {token:i for i,token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

Vocab size: 1130
('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


## Section 2.3 Converting tokens into token IDs

In [6]:
#now build a tokenizer class
class SimpleTokenizerV1:
    def __init__(self, vocab):
        #store vocab as member
        self.str_to_int = vocab
        #create inverse lookup table as member too
        self.int_to_str = {i:s for s,i in vocab.items()}

    #input: text
    #output: vector of ids
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        return [self.str_to_int[token] for token in preprocessed]
        
        
    #input: vector of ids
    #output: text sequence (str)
    def decode(self, ids):
        text = " ".join([self.int_to_str[idv] for idv in ids])
        #for tokens that are punctuation, remove preceeding space
        #substitute space + punctuation for punctuation (\1)
        text = re.sub(r'\s+([,.?!"()\'])', r'\1',text)
        return text

In [7]:
#test the tokenzier class
tokenizer = SimpleTokenizerV1(vocab)
#sample text from book:
text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)

print('Encoding')
print(ids)

print('Decoding')
print(tokenizer.decode(ids))

print('Self-consistency check')
print(tokenizer.decode(tokenizer.encode(text)))

Encoding
[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]
Decoding
" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.
Self-consistency check
" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


## Section 2.4: Special context tokens

In [8]:
#a test:
#test the tokenzier class
tokenizer = SimpleTokenizerV1(vocab)
#give an example with tokens not in vocabulary
text = "Hello, do you like te. Is this-- a test?"
try:
    ids = tokenizer.encode(text)
except KeyError as e:
    print(f'Missing key: {e}')

Missing key: 'Hello'


In [9]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(['<|endoftext|>','<|unk|>'])
vocab = {tokens:idv for idv, tokens in enumerate(all_tokens)}
print('Print new vocab size:',len(vocab.items()))

for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

Print new vocab size: 1132
('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [10]:
#now build a tokenizer class
class SimpleTokenizerV2:
    def __init__(self, vocab):
        #store vocab as member
        self.str_to_int = vocab
        #create inverse lookup table as member too
        self.int_to_str = {i:s for s,i in vocab.items()}

    #input: text
    #output: vector of ids
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else '<|unk|>' for item in preprocessed
        ]
        return [self.str_to_int[token] for token in preprocessed]
        
        
    #input: vector of ids
    #output: text sequence (str)
    def decode(self, ids):
        text = " ".join([self.int_to_str[idv] for idv in ids])
        #for tokens that are punctuation, remove preceeding space
        #substitute space + punctuation for punctuation (\1)
        text = re.sub(r'\s+([,.?!"()\'])', r'\1',text)
        return text

In [11]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

encoded_text = tokenizer.encode(text)
print(encoded_text)

tokenizer.decode(tokenizer.encode(text))

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.
[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

## Section 2.5: Byte Pair Encoding

In [12]:
import importlib
import tiktoken
print('tiktoken version',importlib.metadata.version('tiktoken'))

tiktoken version 0.12.0


In [13]:
tokenizer = tiktoken.get_encoding('gpt2')
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text,allowed_special = {'<|endoftext|>'})
print(integers)

print(tokenizer.decode(integers))

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]
Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


#### Exercise 2.1

In [14]:
ex_text = 'Akwirw ier'
ex_text_enc = tokenizer.encode(ex_text)
ex_text_dec = tokenizer.decode(ex_text_enc)
print(ex_text)

for a in ex_text_enc:
    b = tokenizer.decode([a])
    print(b,'-->',a)

Akwirw ier
Ak --> 33901
w --> 86
ir --> 343
w --> 86
  --> 220
ier --> 959


## Section 2.6 Sliding window

In [15]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [16]:
enc_sample = enc_text[50:]
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f'x: {x}')
print(f'y:     {y}')

for i in range(1,context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context),'--->',tokenizer.decode([desired]))


x: [290, 4920, 2241, 287]
y:     [4920, 2241, 287, 257]
 and --->  established
 and established --->  himself
 and established himself --->  in
 and established himself in --->  a


In [17]:
import torch
print('PyTorch version:', torch.__version__)
from torch.utils.data import Dataset, DataLoader
from typing import List, Tuple, Optional

PyTorch version: 2.9.0


In [18]:
## This class generates multiple sequences from a single piece of text by using a sliding window
## The window size is window_size while the sequence starting positions are offset by stride
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, window_size, stride):
        self.input_ids: List[torch.Tensor] = []
        self.target_ids: List[torch.Tensor] = []

        token_ids = tokenizer.encode(txt, allowed_special={'<|endoftext|>'})
        assert len(token_ids) > window_size, "Number of tokenized inputs must at least be equal to window_size+1"

        for i in range(0,len(token_ids) - window_size, stride):
            input_chunk = token_ids[i:i + window_size]
            target_chunk = token_ids[i+1: i + window_size + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx) -> Tuple[torch.Tensor, torch.Tensor]:
        return self.input_ids[idx], self.target_ids[idx]

In [19]:
## now a function to wrap the creation of the DataLoader
def create_dataloader_v1(txt, batch_size=4, window_size=256, stride=128,
                         shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding('gpt2')
    dataset = GPTDatasetV1(txt,tokenizer,window_size,stride)

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataloader

In [20]:
#now lets test it
with open('the-verdict.txt','r',encoding='utf-8') as f:
    raw_text = f.read()
dataloader = create_dataloader_v1(raw_text,batch_size=1, window_size=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [21]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, window_size=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## Section 2.7 Token Embeddings

In [22]:
input_ids = torch.tensor([2, 3, 5, 1])
vocab_size = 6
output_dim = 3
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [23]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


## Section 2.8 Positional Encoding

In [24]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size,output_dim)

window_size = 4
dataloader = create_dataloader_v1(raw_text, batch_size=8, 
                                  window_size=window_size, stride=window_size, 
                                  shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)

print('Token IDs:\n', inputs)
print('\nInputs shape:\n',inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [25]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)
#should be batch size x window size (seq. length) x output dimension (feature vector)

torch.Size([8, 4, 256])


In [26]:
#Introduce another embedding layer for the positional encoding
context_length = window_size
pos_embedding_layer = torch.nn.Embedding(context_length,output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)
print(pos_embeddings)
#should be context_length x output dimension

torch.Size([4, 256])
tensor([[ 1.7375, -0.5620, -0.6303,  ..., -0.2277,  1.5748,  1.0345],
        [ 1.6423, -0.7201,  0.2062,  ...,  0.4118,  0.1498, -0.4628],
        [-0.4651, -0.7757,  0.5806,  ...,  1.4335, -0.4963,  0.8579],
        [-0.6754, -0.4628,  1.4323,  ...,  0.8139, -0.7088,  0.4827]],
       grad_fn=<EmbeddingBackward0>)


In [27]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
